In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as LA
%matplotlib inline

# Set custom colors
colors = [['#cf4c34','#078752','#aaaaaa','#6333ed','#d6ca54','#0ca0ad','#2ea62a','#c96b0d','#d947bb'],\
          ['#4C1C13','#073824','#555555','#3712A0','#000000','#000000','#000000','#000000','#000000'],\
          ['#FF745A','#0BD480','#cccccc','#8D66FF','#EEEEEE','#EEEEEE','#EEEEEE','#EEEEEE','#EEEEEE']]

# Set plot fonts
plt.rcParams['font.sans-serif'] = "Gill Sans MT"
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.size'] = 12
plt.rcParams['font.weight'] = 100

ModuleNotFoundError: No module named 'numpy'

In [2]:
launchpadBasePosition = np.array([26.51615, 17.572+1.5, 20.04752])
launchpadOffA = np.array([-2.4,0.0,0.0])
launchpadOffB = np.array([0.0,0.0,2.4])

goalshelfBasePosition = np.array([26.57, 20.41, 1.93])
goalshelfOffA = np.array([-2.26,0.0,0.0])
goalshelfOffB = np.array([0.0, 1.70,0.0])

def goalshelfPos(shelfIndexA,shelfIndexB):
    return(goalshelfBasePosition + goalshelfOffA*shelfIndexA + goalshelfOffB*shelfIndexB )

def launchpadPos(launchpadIndexA,launchpadIndexB):
    return(launchpadBasePosition + launchpadOffA*launchpadIndexA + launchpadOffB*launchpadIndexB )

In [3]:
DRONE_SPEED = 0.018*50 # Speed in units per second (instead of units per frame)
# DRONE_SPEED = 0.018
# DRONE_SPEED = 1.8
TAKEOFF_HOVERING_TIME = 1.5/DRONE_SPEED

class Flight:
    def __init__(self,start_time,startpoint_indices,endpoint_indices):
        self.speed = DRONE_SPEED
        self.endpoint = goalshelfPos(startpoint_indices[0],startpoint_indices[1])
        self.startpoint = launchpadPos(startpoint_indices[0],startpoint_indices[1])
        self.start_time = start_time
        self.followedBy = False
        self.eventID = -1
        
        self.collidesWith = -1
        self.collidesAt = -1
        
        self.endIndices = endpoint_indices
        self.startIndices = startpoint_indices
        
        distance = self.endpoint-self.startpoint
        #print(distance)
        self.flight_vector = (distance/LA.norm(distance))*self.speed
        # calculate the position the flight would be at at time zero
        # rather than its unique start time
        self.zeroPosition = self.startpoint - start_time*self.flight_vector
        
    def position(self,time):
        flightposition = time*self.flight_vector + self.zeroPosition
        return(flightposition)
    
    def closestDistanceTo(self,other):
        # the difference between the two lines:
        #     w_self(t) = a*t + b
        #     w_other(t) = c*t + d
        # (where a,b,c,d are vectors in 3-space, and t is the time parameter) will be :
        #     w_diff(t) = w_self(t) - w_other(t) = (a-c)*t + (b-d)
        # we want to find when this difference distance minimizes the two-norm:
        #     min_t \|w_diff(t) \|^2 = w_diff^T * w_diff
        # the critical point will be when the derivative of the objective is zero:
        #     0 = (a-c)^T * w_diff(t) = (a-c)^T (a-c) * t + (a-c)^T (b-d)
        # which solves for the time parameter t as:
        #     t = -<a-c, b-d> / \|a-c\|^2
        difference_flight_vector = self.flight_vector - other.flight_vector
        difference_zeroPosition = self.zeroPosition - other.zeroPosition
        differenceMagnitude = np.dot(difference_flight_vector,difference_flight_vector)
        # if the two flights are parallel than all points are equally close
        if(differenceMagnitude == 0):
            closestTime = 0
            # the difference in the flight vectors will be zero for parallel lines
            # which would cause divide by zero issues in finding a unique closest time
        else:
            closestTime = -np.dot(difference_flight_vector,difference_zeroPosition) / differenceMagnitude
        # Don't count approach times past the time limit
        if(closestTime > 180):
            closestTime = 180
        return(self.position(closestTime) - other.position(closestTime), closestTime)
    
    def endTime(self):
        distance = self.endpoint - self.startpoint
        return(self.start_time + LA.norm(distance)/self.speed + TAKEOFF_HOVERING_TIME)
    
    def __repr__(self):
        return repr((self.startpoint,self.endpoint))
    

In [4]:
COLLISION_RADIUS = 1.4

In [5]:
def generateAcceptablePath(nextTakeoff,drones,collision_tally):
    takingOffDrone = nextTakeoff.drone
    otherDrones = [drone for drone in drones if drone!=takingOffDrone]
    # Generate an approved flight
    approval = False
    path_collides = 0
    approval_attempts = 0
    while approval == False:
        newFlight = Flight(nextTakeoff.time, [random.randint(0,9),random.randint(0,2)],[random.randint(0,9),random.randint(0,3)] )
        newFlight.drone = takingOffDrone
        approval = True
        # Check for collision
        path_collides = 0
        for drone in otherDrones:
            if(drone.flightplan):
                approach, approachtime = drone.flightplan[-1].closestDistanceTo(newFlight)
                #print(LA.norm(approach))
                if(LA.norm(approach) < COLLISION_RADIUS):
                    if(collision_tally < min(newFlight.endTime(),180)/10): # enforce that the number of collisions that have occured by the n-tieth second is less than n
                        if(path_collides==0 and drone.flightplan[-1].collidesWith==-1): # enforce that each path never collides with more than one other path
                            path_collides = 1
                            newFlight.collidesWith = drone.flightplan[-1]
                            newFlight.collidesAt = newFlight.position(approachtime)
                            drone.flightplan[-1].collidesWith = newFlight
                            drone.flightplan[-1].collidesAt = drone.flightplan[-1].position(approachtime)
                        else:
                            approval = False
                    else:
                        approval = False
        approval_attempts += 1
        assert(approval_attempts < 2000),"Couldn't plan safely"
    return(newFlight,path_collides)

In [6]:
class Drone:
    def __init__(self,name,number):
        self.flightplan = []
        self.name = name
        self.IDnumber = number
        
    def newTakeoff(self,flight):
        self.flightplan.append(flight)
        
    def __repr__(self):
        return repr((self.name))

In [7]:
import random
NUM_DRONES = 10

class Takeoff:
    def __init__(self,time,drone):
        self.time = time
        self.drone = drone
        
    def __repr__(self):
        return repr((self.time, self.drone))

drones = []
upcomingTakeoffs = []
collision_tally = 0
curEventID = 0

for ii in range(NUM_DRONES):
    newDrone = Drone("Drone #"+str(ii),ii)
    drones.append(newDrone)
    launch = Takeoff(0.0+ii*10.0/NUM_DRONES,newDrone)
    upcomingTakeoffs.append( launch )
    #newFlight, path_collides = generateAcceptablePath(launch,drones,collision_tally)

#print("initial takeoff created "+str(collision_tally)+" collisions")

# Pick out the first respawn takeoff
upcomingTakeoffs = sorted(upcomingTakeoffs, key=lambda takeoff: takeoff.time)
nextTakeoff = upcomingTakeoffs.pop(0)
takeoffOrderedFlights = []

while nextTakeoff.time < 180:
    takingOffDrone = nextTakeoff.drone
    newFlight, path_collides = generateAcceptablePath(nextTakeoff,drones,collision_tally)
    collision_tally += path_collides
    # Record in takeoff history
    newFlight.eventID = curEventID
    curEventID += 1
    takeoffOrderedFlights.append(newFlight)
    # Assign this approved flight to the drone
    if(takingOffDrone.flightplan):
        takingOffDrone.flightplan[-1].followedBy = newFlight
    takingOffDrone.newTakeoff(newFlight)
    # Queue up the following takeoff once this new flight terminates
    upcomingTakeoffs.append(Takeoff(newFlight.endTime(),takingOffDrone))
    # Sort the takeoffs to identify who is taking off next
    upcomingTakeoffs = sorted(upcomingTakeoffs, key=lambda takeoff: takeoff.time)
    nextTakeoff = upcomingTakeoffs.pop(0)
    
print("done with "+str(collision_tally)+" collisions")
# Show me the flight plan for a drone
drones[0].flightplan

done with 18 collisions


[(array([ 4.91615, 19.072  , 24.84752]), array([ 6.23, 23.81,  1.93])),
 (array([14.51615, 19.072  , 24.84752]), array([15.27, 23.81,  1.93])),
 (array([ 7.31615, 19.072  , 22.44752]), array([ 8.49, 22.11,  1.93])),
 (array([16.91615, 19.072  , 20.04752]), array([17.53, 20.41,  1.93])),
 (array([ 9.71615, 19.072  , 20.04752]), array([10.75, 20.41,  1.93])),
 (array([12.11615, 19.072  , 20.04752]), array([13.01, 20.41,  1.93])),
 (array([ 4.91615, 19.072  , 20.04752]), array([ 6.23, 20.41,  1.93])),
 (array([12.11615, 19.072  , 24.84752]), array([13.01, 23.81,  1.93]))]

In [8]:
import csv
with open('flightplan.csv', 'w') as csvfile:
    planwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    planwriter.writerow(['eventID', 'droneID', 'startingLaunchpadIndexX', 'startingLaunchpadIndexY',
                         'shelfIndexX', 'shelfIndexY', 'followedBy',
                         'teleportToLaunchpadIndexX', 'teleportToLaunchpadIndexY', 'collidesWithNum', 'collidesAtPos'])
    for flight in takeoffOrderedFlights:
        collidesWithReport = ""
        if(flight.collidesWith == -1):
            collidesWithReport = [-2,-2]
        else:
            collidesWithReport = [flight.collidesWith.drone.IDnumber,flight.collidesAt]
        if(flight.followedBy):
            planwriter.writerow([flight.eventID, flight.drone.IDnumber] + flight.startIndices + flight.endIndices
                                + [flight.followedBy.eventID] + flight.followedBy.startIndices
                                + collidesWithReport)
        else:
            planwriter.writerow([flight.eventID, flight.drone.IDnumber] + flight.startIndices + flight.endIndices
                                + [-1] + [-1,-1]
                                + collidesWithReport)

In [13]:
def goalshelfPos(shelfIndexA,shelfIndexB):
    return(goalshelfBasePosition + goalshelfOffA*shelfIndexA + goalshelfOffB*shelfIndexB )

def launchpadPos(launchpadIndexA,launchpadIndexB):
    return(launchpadBasePosition + launchpadOffA*launchpadIndexA + launchpadOffB*launchpadIndexB )

total_distances = 0.0
total_arcs = 0.0
for startX in range(0,9):
    for startY in range(0,2):
        for endX in range(0,9):
            for endY in range(0,3):
                total_arcs += 1.0
                total_distances += LA.norm(goalshelfPos(startX,startY) - launchpadPos(endX,endY))

average_trip_distance = total_distances/total_arcs
average_trip_time = average_trip_distance/DRONE_SPEED
print(average_trip_time)

24.757130150754456
